Dependencies

In [1]:
! pip install keras
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tqdm.keras import TqdmCallback
from sklearn.preprocessing import StandardScaler

Upload the data

In [3]:
# We take the document when we have all our data normalized:
df_15m = pd.read_csv('/content/sample_data/Full_data_15m.csv')

In [4]:
df_15m.head()

,Date_15m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,collapse,ATH,grow,rise,fall,date,open,close,Return,Return_sign
0,2022-01-01 00:15:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.816322,0.632644,0.342175,0.161173,11,4,...,0,9,6,7,4,2022-01-01 00:15:00,46332.52,46375.42,0.000926,1
1,2022-01-01 00:30:00,"['2022-01-01 00:15:01', '2022-01-01 00:15:05',...",The current price of Bitcoin is $46453.00.\nTh...,big_blockers\nbmurphypointman\nButterLemonBot\...,0.716851,0.433702,0.305638,0.136947,9,42,...,1,8,1,6,38,2022-01-01 00:30:00,46375.42,46610.81,0.005076,1
2,2022-01-01 00:45:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.725069,0.450138,0.438272,0.172915,9,15,...,0,6,2,5,17,2022-01-01 00:45:00,46610.81,46656.13,0.000972,1
3,2022-01-01 01:00:00,"['2022-01-01 00:45:02', '2022-01-01 00:45:04',...","#Bitcoin $BTC \nPrice: $46,630.10\n\nAreas of ...",DCA_Cryptoz\nbidcoindad\ncoinok\nbmurphypointm...,0.756357,0.512714,0.381818,0.178866,5,11,...,1,5,1,2,6,2022-01-01 01:00:00,46656.14,46766.99,0.002376,1
4,2022-01-01 01:15:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.770306,0.540612,0.343195,0.143037,6,13,...,0,6,8,4,13,2022-01-01 01:15:00,46766.99,46815.33,0.001034,1


In [5]:
df_15m.columns

Index(['Date_15m', 'datetime', 'text', 'username', 'kk08_pos_score',
       'kk08_score_diff', 'Elkulako_label', 'Sentiment_vader', 'pred',
       'analysis', 'short', 'long', 'buy', 'sell', 'bull', 'bear', 'bankrupt',
       'fraud', 'cbdc', 'record', 'crisis', 'collapse', 'ATH', 'grow', 'rise',
       'fall', 'date', 'open', 'close', 'Return', 'Return_sign'],
      dtype='object')

In [6]:
# I just colect the information i will need to use to train the NN for each of the sentiment results:
df_15m_Vader = df_15m[['Sentiment_vader','Return_sign']]
df_15m_kk08 = df_15m[['kk08_pos_score','Return_sign']]
df_15m_Elkulako = df_15m[['Elkulako_label','Return_sign']]


We define the model we want to use

In [7]:
#model = Sequential([
    #Dense(32, activation='relu', input_shape=(1,)),
    #Dropout(0.3),
    #Dense(16, activation='relu'),
    #Dropout(0.3),
    #Dense(8, activation='relu'),
    #Dense(1, activation='sigmoid')
#])

In [8]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [9]:
# Define callbacks to stop and dont overfit the model
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

## KK08

In [10]:
df_15m_kk08.columns

Index(['kk08_pos_score', 'Return_sign'], dtype='object')

In [11]:
X_kk08 = df_15m_kk08 ['kk08_pos_score']
Y_kk08 = df_15m_kk08 ['Return_sign']
#Y_kk08 = np.array(df_15m_kk08 ['Return_sign'])

In [12]:
train_share = 0.85
df_x_1_train = X_kk08.iloc[:int(len(X_kk08)*train_share)].reset_index(drop=True)
df_y_train = Y_kk08.iloc[:int(len(Y_kk08)*train_share)].reset_index(drop=True)

df_x_1_test = X_kk08.iloc[int(len(X_kk08)*train_share):].reset_index(drop=True)
df_y_test = Y_kk08.iloc[int(len(Y_kk08)*train_share):].reset_index(drop=True)

In [13]:
# Checking that class distribution is more or less balance in both dataset.
print("Training set class distribution:")
print(pd.Series(df_y_train).value_counts())
print("Test set class distribution:")
print(pd.Series(df_y_test).value_counts())

Training set class distribution:
Return_sign
0    1318
1    1211
Name: count, dtype: int64
Test set class distribution:
Return_sign
1    236
0    211
Name: count, dtype: int64


In [14]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_train, X_valid = df_x_1_train[:split_index], df_x_1_train[split_index:]
y_train, y_valid = df_y_train[:split_index], df_y_train[split_index:]

In [15]:
# Since the first result was not completely satisfactory, we try to improve the solution proving adding or remove layers of a NN to se how we can improve the NN:
def create_model(num_layers=6, neurons_per_layer=48, input_shape=(10,), output_units=1, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons_per_layer, activation=activation, input_shape=input_shape))

    # Agregar las capas ocultas especificadas
    for _ in range(num_layers - 1):
        model.add(Dense(neurons_per_layer, activation=activation))

    # Capa de salida
    model.add(Dense(output_units, activation='sigmoid'))  # Cambia 'sigmoid' a 'softmax' si tienes una clasificación multiclase

    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Definir parámetros de la red
num_layers_list = [2, 3, 4,6,8]  # Diferentes números de capas a probar
neurons_per_layer_list = [16, 32, 64,128]  # Diferentes números de neuronas en cada capa

In [16]:
# Initialice the variables we need to keep track of the model:
best_accuracy = 0
best_roc_auc = 0
best_config = {}

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nTraining the model with {num_layers} hidden layers y {neurons_per_layer} neurons per layer")

        model = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(1,))

        # We define an early stop to avoid overfitting:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


        history = model.fit( df_x_1_train, df_y_train,validation_data=(X_valid,  y_valid), epochs=200, batch_size=16, callbacks=[early_stopping, TqdmCallback(verbose=1)], verbose=0)

        # Loss and Accuracy :
        loss, accuracy = model.evaluate(df_x_1_test, df_y_test, verbose=0)
        print(f"Layers: {num_layers}, Neurons: {neurons_per_layer} -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        # ROC AUC
        y_pred_probs = model.predict(df_x_1_test)
        roc_auc = roc_auc_score(df_y_test, y_pred_probs)
        print(f"ROC AUC: {roc_auc:.4f}")

        # Check if we are handeling the best configuration:
        if accuracy > best_accuracy or (accuracy == best_accuracy and roc_auc > best_roc_auc):
            best_accuracy = accuracy
            best_roc_auc = roc_auc
            best_config = {
                'num_layers': num_layers,
                'neurons_per_layer': neurons_per_layer,
                'accuracy': best_accuracy,
                'loss': loss,
                'roc_auc': best_roc_auc
            }



Training the model with 2 hidden layers y 16 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Layers: 2, Neurons: 16 -> Loss: 0.6954, Accuracy: 0.4720
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
ROC AUC: 0.5426

Training the model with 2 hidden layers y 32 neurons per layer


...

Layers: 8, Neurons: 64 -> Loss: 0.6945, Accuracy: 0.4720
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ROC AUC: 0.5439

Training the model with 8 hidden layers y 128 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Layers: 8, Neurons: 128 -> Loss: 0.6937, Accuracy: 0.4720
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
ROC AUC: 0.5426


In [17]:
# Show the best metrics and configuration of the Neuronal Network :
print("\nBetter configuration finded:")
print(f"Hidden Layers: {best_config['num_layers']}")
print(f"Neuron(s) per layer: {best_config['neurons_per_layer']}")
print(f"Accuracy: {best_config['accuracy']:.4f}")
print(f"Loss: {best_config['loss']:.4f}")
print(f"ROC AUC: {best_config['roc_auc']:.4f}")


Better configuration finded:
Hidden Layers: 4
Neuron(s) per layer: 64
Accuracy: 0.5280
Loss: 0.6927
ROC AUC: 0.4575


## With Lag

In [18]:
df_15m['kk08_pos_score_lag1']=df_15m['kk08_pos_score'].shift(1)
df_15m['kk08_pos_score_lag2']=df_15m['kk08_pos_score'].shift(2)
df_15m['kk08_pos_score_lag3']=df_15m['kk08_pos_score'].shift(3)

In [19]:
df_x_2 = df_15m.dropna().reset_index(drop=True)[["kk08_pos_score", "kk08_pos_score_lag1", "kk08_pos_score_lag2", "kk08_pos_score_lag3"]]
df_y_2 = df_15m["Return_sign"].iloc[3:].reset_index(drop=True)

In [20]:
train_share = 0.85
df_x_2_train = df_x_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)
df_y_2_train = df_y_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)

df_x_2_test = df_x_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)
df_y_2_test = df_y_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)

In [21]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_2train, X_2valid = df_x_2_train[:split_index], df_x_2_train[split_index:]
y_2train, y_2valid = df_y_2_train[:split_index], df_y_2_train[split_index:]

In [22]:
# Initialice the variables we need to keep track of the model:
best_accuracy = 0
best_roc_auc = 0
best_config = {}

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nTraining the model with {num_layers} hidden layers and {neurons_per_layer} neurons per layer")

        model_lag = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(4,))

        # We define an early stop to avoid overfitting:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


        history = model_lag.fit( df_x_2_train, df_y_2_train,validation_data=(X_2valid, y_2valid), epochs=200, batch_size=16, callbacks=[early_stopping, TqdmCallback(verbose=1)], verbose=0)

        # Loss and Accuracy :
        loss, accuracy = model_lag.evaluate(df_x_2_test, df_y_2_test, verbose=0)
        print(f"Capas: {num_layers}, Neuronas: {neurons_per_layer} -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        # ROC AUC
        y_pred_probs = model_lag.predict(df_x_2_test)
        roc_auc = roc_auc_score( df_y_2_test, y_pred_probs)
        print(f"ROC AUC: {roc_auc:.4f}")

        # Check if we are handeling the best configuration:
        if accuracy > best_accuracy or (accuracy == best_accuracy and roc_auc > best_roc_auc):
            best_accuracy = accuracy
            best_roc_auc = roc_auc
            best_config = {
                'num_layers': num_layers,
                'neurons_per_layer': neurons_per_layer,
                'accuracy': best_accuracy,
                'loss': loss,
                'roc_auc': best_roc_auc
            }



Training the model with 2 hidden layers and 16 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 2, Neuronas: 16 -> Loss: 0.6941, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC AUC: 0.4523

Training the model with 2 hidden layers and 32 neurons per layer


...

Capas: 8, Neuronas: 64 -> Loss: 0.6947, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
ROC AUC: 0.5299

Training the model with 8 hidden layers and 128 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 8, Neuronas: 128 -> Loss: 0.6937, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ROC AUC: 0.4901


In [23]:
# Show the best metrics and configuration of the Neuronal Network :
print("\nBetter configuration finded:")
print(f"Hidden Layers: {best_config['num_layers']}")
print(f"Neuron(s) per layer: {best_config['neurons_per_layer']}")
print(f"Accuracy: {best_config['accuracy']:.4f}")
print(f"Loss: {best_config['loss']:.4f}")
print(f"ROC AUC: {best_config['roc_auc']:.4f}")


Better configuration finded:
Hidden Layers: 6
Neuron(s) per layer: 16
Accuracy: 0.4731
Loss: 0.6950
ROC AUC: 0.5326


## VADER

In [24]:
df_15m_Vader.columns

Index(['Sentiment_vader', 'Return_sign'], dtype='object')

In [25]:
X_Vadder = df_15m_Vader ['Sentiment_vader']
Y_Vadder = df_15m_Vader ['Return_sign']

In [26]:
train_share = 0.85
X_train_vadder = X_Vadder.iloc[:int(len(X_Vadder)*train_share)].reset_index(drop=True)
Y_train_vadder= Y_Vadder.iloc[:int(len(Y_Vadder)*train_share)].reset_index(drop=True)

X_test_vadder= X_Vadder.iloc[int(len(X_Vadder)*train_share):].reset_index(drop=True)
Y_test_vadder= Y_Vadder.iloc[int(len(Y_Vadder)*train_share):].reset_index(drop=True)

In [27]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_train_V, X_valid_V = X_train_vadder[:split_index], X_train_vadder[split_index:]
y_train_V, y_valid_V = Y_train_vadder[:split_index], Y_train_vadder[split_index:]

In [28]:
# Initialice the variables we need to keep track of the model:
best_accuracy = 0
best_roc_auc = 0
best_config = {}

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nEntrenando modelo con {num_layers} capas ocultas y {neurons_per_layer} neuronas por capa")

        model = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(1,))

        # We define an early stop to avoid overfitting:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


        history = model.fit(X_test_vadder , Y_test_vadder,validation_split=0.2, epochs=200, batch_size=16, callbacks=[early_stopping, TqdmCallback(verbose=1)], verbose=0)

        # Loss and Accuracy :
        loss, accuracy = model.evaluate(X_test_vadder, Y_test_vadder, verbose=0)
        print(f"Capas: {num_layers}, Neuronas: {neurons_per_layer} -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        # ROC AUC
        y_pred_probs = model.predict(X_test_vadder).ravel()
        roc_auc = roc_auc_score(Y_test_vadder, y_pred_probs)
        print(f"ROC AUC: {roc_auc:.4f}")

        # Check if we are handeling the best configuration:
        if accuracy > best_accuracy or (accuracy == best_accuracy and roc_auc > best_roc_auc):
            best_accuracy = accuracy
            best_roc_auc = roc_auc
            best_config = {
                'num_layers': num_layers,
                'neurons_per_layer': neurons_per_layer,
                'accuracy': best_accuracy,
                'loss': loss,
                'roc_auc': best_roc_auc
            }




Entrenando modelo con 2 capas ocultas y 16 neuronas por capa


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 2, Neuronas: 16 -> Loss: 0.6921, Accuracy: 0.5280
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC AUC: 0.5121

Entrenando modelo con 2 capas ocultas y 32 neuronas por capa


...

Capas: 8, Neuronas: 64 -> Loss: 0.6922, Accuracy: 0.5280
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ROC AUC: 0.5060

Entrenando modelo con 8 capas ocultas y 128 neuronas por capa


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 8, Neuronas: 128 -> Loss: 0.6924, Accuracy: 0.5280
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
ROC AUC: 0.5122


In [29]:
# Show the best metrics and configuration of the Neuronal Network :
print("\nBetter configuration finded:")
print(f"Hidden Layers: {best_config['num_layers']}")
print(f"Neuron(s) per layer: {best_config['neurons_per_layer']}")
print(f"Accuracy: {best_config['accuracy']:.4f}")
print(f"Loss: {best_config['loss']:.4f}")
print(f"ROC AUC: {best_config['roc_auc']:.4f}")


Better configuration finded:
Hidden Layers: 8
Neuron(s) per layer: 128
Accuracy: 0.5280
Loss: 0.6924
ROC AUC: 0.5122


### With Lag

In [30]:
df_15m['Vader_lag1']=df_15m['Sentiment_vader'].shift(1)
df_15m['Vader_lag2']=df_15m['Sentiment_vader'].shift(2)
df_15m['Vader_lag3']=df_15m['Sentiment_vader'].shift(3)

In [31]:
X_train_vadder_lag = df_15m.dropna().reset_index(drop=True)[['Sentiment_vader', 'Vader_lag1', 'Vader_lag2', 'Vader_lag3']]
Y_train_vadder_lag= df_15m["Return_sign"].iloc[3:].reset_index(drop=True)

In [32]:
X_train_vadder_lag

,Sentiment_vader,Vader_lag1,Vader_lag2,Vader_lag3
0,0.178866,0.172915,0.136947,0.161173
1,0.143037,0.178866,0.172915,0.136947
2,0.168293,0.143037,0.178866,0.172915
3,0.201061,0.168293,0.143037,0.178866
4,0.199579,0.201061,0.168293,0.143037
...,...,...,...,...
2968,0.167436,0.171507,0.120725,0.163388
2969,0.161682,0.167436,0.171507,0.120725
2970,0.161257,0.161682,0.167436,0.171507
2971,0.110640,0.161257,0.161682,0.167436


In [33]:
train_share = 0.85
X_train_vadder_2 = X_train_vadder_lag.iloc[:int(len(X_train_vadder_lag)*train_share)].reset_index(drop=True)
Y_train_vadder_2 = Y_train_vadder_lag.iloc[:int(len(X_train_vadder_lag)*train_share)].reset_index(drop=True)

X_test_vadder_2 = X_train_vadder_lag.iloc[int(len(X_train_vadder_lag)*train_share):].reset_index(drop=True)
Y_test_vadder_2 = Y_train_vadder_lag.iloc[int(len(Y_train_vadder_lag)*train_share):].reset_index(drop=True)



In [34]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_2train, X_2valid = X_train_vadder_2[:split_index], X_train_vadder_2[split_index:]
y_2train, y_2valid = Y_train_vadder_2[:split_index], Y_train_vadder_2[split_index:]

In [35]:
model_lag = Sequential([
    Dense(32, activation='relu', input_shape=(4,)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model_lag.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
print("Shape of df_x_2_train:", X_train_vadder_2.shape)
print("Shape of df_y_2_train:", Y_train_vadder_2.shape)

Shape of df_x_2_train: (2527, 4)
Shape of df_y_2_train: (2527,)


In [39]:
# Initialice the variables we need to keep track of the model:
best_accuracy = 0
best_roc_auc = 0
best_config = {}

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nTraining the model with {num_layers} hidden layers and {neurons_per_layer} neurons per layer")

        model_lag = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(4,))

        # We define an early stop to avoid overfitting:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


        history = model_lag.fit( X_train_vadder_2, Y_train_vadder_2,validation_data=(X_2valid, y_2valid), epochs=200, batch_size=16, callbacks=[early_stopping, TqdmCallback(verbose=1)], verbose=0)

        # Loss and Accuracy :
        loss, accuracy = model_lag.evaluate(X_test_vadder_2, Y_test_vadder_2, verbose=0)
        print(f"Capas: {num_layers}, Neuronas: {neurons_per_layer} -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        # ROC AUC
        y_pred_probs = model_lag.predict(X_test_vadder_2)
        roc_auc = roc_auc_score( Y_test_vadder_2, y_pred_probs)
        print(f"ROC AUC: {roc_auc:.4f}")

        # Check if we are handeling the best configuration:
        if accuracy > best_accuracy or (accuracy == best_accuracy and roc_auc > best_roc_auc):
            best_accuracy = accuracy
            best_roc_auc = roc_auc
            best_config = {
                'num_layers': num_layers,
                'neurons_per_layer': neurons_per_layer,
                'accuracy': best_accuracy,
                'loss': loss,
                'roc_auc': best_roc_auc
            }

# Show the best metrics and configuration of the Neuronal Network :
print("\nBetter configuration finded:")
print(f"Hidden Layers: {best_config['num_layers']}")
print(f"Neuron(s) per layer: {best_config['neurons_per_layer']}")
print(f"Accuracy: {best_config['accuracy']:.4f}")
print(f"Loss: {best_config['loss']:.4f}")
print(f"ROC AUC: {best_config['roc_auc']:.4f}")


Training the model with 2 hidden layers and 16 neurons per layer


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 2, Neuronas: 16 -> Loss: 0.6941, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
ROC AUC: 0.4960

Training the model with 2 hidden layers and 32 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Capas: 8, Neuronas: 64 -> Loss: 0.6948, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
ROC AUC: 0.5035

Training the model with 8 hidden layers and 128 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 8, Neuronas: 128 -> Loss: 0.6940, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ROC AUC: 0.4588

Better configuration finded:
Hidden Layers: 8
Neuron(s) per layer: 16
Accuracy: 0.4731
Loss: 0.6954
ROC AUC: 0.5311


## Elkulako

In [40]:
df_15m_Elkulako.columns

Index(['Elkulako_label', 'Return_sign'], dtype='object')

In [41]:
X_Elkulako = df_15m_Elkulako ['Elkulako_label']
Y_Elkulako = df_15m_Elkulako ['Return_sign']

In [42]:
train_share = 0.85
X_train_Elkulako = X_Elkulako.iloc[:int(len(X_Elkulako)*train_share)].reset_index(drop=True)
Y_train_Elkulako= Y_Elkulako.iloc[:int(len(Y_Elkulako)*train_share)].reset_index(drop=True)

X_test_Elkulako= X_Vadder.iloc[int(len(X_Elkulako)*train_share):].reset_index(drop=True)
Y_test_Elkulako= Y_Vadder.iloc[int(len(Y_Elkulako)*train_share):].reset_index(drop=True)

In [43]:
train_share = 0.90
split_index = int(train_share * len(df_x_1_train))
X_train_V, X_valid_V = X_train_Elkulako [:split_index], X_train_Elkulako [split_index:]
y_train_V, y_valid_V = Y_train_Elkulako [:split_index], Y_train_Elkulako [split_index:]

We train the model

In [45]:
# Initialice the variables we need to keep track of the model:
best_accuracy = 0
best_roc_auc = 0
best_config = {}

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nEntrenando modelo con {num_layers} capas ocultas y {neurons_per_layer} neuronas por capa")

        model = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(1,))

        # We define an early stop to avoid overfitting:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


        history = model.fit(X_test_Elkulako , Y_test_Elkulako,validation_split=0.2, epochs=200, batch_size=16, callbacks=[early_stopping, TqdmCallback(verbose=1)], verbose=0)

        # Loss and Accuracy :
        loss, accuracy = model.evaluate(X_test_vadder, Y_test_vadder, verbose=0)
        print(f"Capas: {num_layers}, Neuronas: {neurons_per_layer} -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        # ROC AUC
        y_pred_probs = model.predict(X_test_Elkulako).ravel()
        roc_auc = roc_auc_score(Y_test_Elkulako, y_pred_probs)
        print(f"ROC AUC: {roc_auc:.4f}")

        # Check if we are handeling the best configuration:
        if accuracy > best_accuracy or (accuracy == best_accuracy and roc_auc > best_roc_auc):
            best_accuracy = accuracy
            best_roc_auc = roc_auc
            best_config = {
                'num_layers': num_layers,
                'neurons_per_layer': neurons_per_layer,
                'accuracy': best_accuracy,
                'loss': loss,
                'roc_auc': best_roc_auc
            }

# Show the best metrics and configuration of the Neuronal Network :
print("\nBetter configuration finded:")
print(f"Hidden Layers: {best_config['num_layers']}")
print(f"Neuron(s) per layer: {best_config['neurons_per_layer']}")
print(f"Accuracy: {best_config['accuracy']:.4f}")
print(f"Loss: {best_config['loss']:.4f}")
print(f"ROC AUC: {best_config['roc_auc']:.4f}")


Entrenando modelo con 2 capas ocultas y 16 neuronas por capa


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 2, Neuronas: 16 -> Loss: 0.6931, Accuracy: 0.5078
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC AUC: 0.4883

Entrenando modelo con 2 capas ocultas y 32 neuronas por capa


...

Capas: 8, Neuronas: 64 -> Loss: 0.6925, Accuracy: 0.5280
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ROC AUC: 0.5121

Entrenando modelo con 8 capas ocultas y 128 neuronas por capa


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 8, Neuronas: 128 -> Loss: 0.6921, Accuracy: 0.5280
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
ROC AUC: 0.5118

Better configuration finded:
Hidden Layers: 8
Neuron(s) per layer: 64
Accuracy: 0.5280
Loss: 0.6925
ROC AUC: 0.5121


### With Lag

In [46]:
df_15m['Elkulako_lag1']=df_15m['Elkulako_label'].shift(1)
df_15m['Elkulako_lag2']=df_15m['Elkulako_label'].shift(2)
df_15m['Elkulako_lag3']=df_15m['Elkulako_label'].shift(3)

In [47]:
X_train_Elkulako_lag = df_15m.dropna().reset_index(drop=True)[['Elkulako_label', 'Elkulako_lag1', 'Elkulako_lag2', 'Elkulako_lag3']]
Y_train_Elkulako_lag= df_15m["Return_sign"].iloc[3:].reset_index(drop=True)

In [48]:
train_share = 0.85
X_train_Elkulako_2 = X_train_Elkulako_lag.iloc[:int(len(X_train_Elkulako_lag)*train_share)].reset_index(drop=True)
Y_train_Elkulako_2 = Y_train_vadder_lag.iloc[:int(len(X_train_Elkulako_lag)*train_share)].reset_index(drop=True)

X_test_Elkulako_2 = X_train_Elkulako_lag.iloc[int(len(X_train_Elkulako_lag)*train_share):].reset_index(drop=True)
Y_test_Elkulako_2 = Y_train_Elkulako_lag.iloc[int(len(Y_train_Elkulako_lag)*train_share):].reset_index(drop=True)



In [49]:
train_share = 0.90
split_index = int(train_share * len( X_train_Elkulako_2))
X_2train, X_2valid = X_train_Elkulako_2[:split_index], X_train_Elkulako_2[split_index:]
y_2train, y_2valid = Y_train_Elkulako_2[:split_index], Y_train_Elkulako_2[split_index:]

In [50]:
Y_test_Elkulako_2.shape

(446,)

In [51]:
# Initialice the variables we need to keep track of the model:
best_accuracy = 0
best_roc_auc = 0
best_config = {}

# Probar diferentes configuraciones de capas y neuronas
for num_layers in num_layers_list:
    for neurons_per_layer in neurons_per_layer_list:
        print(f"\nTraining the model with {num_layers} hidden layers and {neurons_per_layer} neurons per layer")

        model_lag = create_model(num_layers=num_layers, neurons_per_layer=neurons_per_layer, input_shape=(4,))

        # We define an early stop to avoid overfitting:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


        history = model_lag.fit( X_train_Elkulako_2, Y_train_Elkulako_2,validation_data=(X_2valid, y_2valid), epochs=200, batch_size=16, callbacks=[early_stopping, TqdmCallback(verbose=1)], verbose=0)

        # Loss and Accuracy :
        loss, accuracy = model_lag.evaluate(X_test_Elkulako_2, Y_test_Elkulako_2, verbose=0)
        print(f"Capas: {num_layers}, Neuronas: {neurons_per_layer} -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        # ROC AUC
        y_pred_probs = model_lag.predict(X_test_Elkulako_2)
        roc_auc = roc_auc_score( Y_test_Elkulako_2, y_pred_probs)
        print(f"ROC AUC: {roc_auc:.4f}")

        # Check if we are handeling the best configuration:
        if accuracy > best_accuracy or (accuracy == best_accuracy and roc_auc > best_roc_auc):
            best_accuracy = accuracy
            best_roc_auc = roc_auc
            best_config = {
                'num_layers': num_layers,
                'neurons_per_layer': neurons_per_layer,
                'accuracy': best_accuracy,
                'loss': loss,
                'roc_auc': best_roc_auc
            }

# Show the best metrics and configuration of the Neuronal Network :
print("\nBetter configuration finded:")
print(f"Hidden Layers: {best_config['num_layers']}")
print(f"Neuron(s) per layer: {best_config['neurons_per_layer']}")
print(f"Accuracy: {best_config['accuracy']:.4f}")
print(f"Loss: {best_config['loss']:.4f}")
print(f"ROC AUC: {best_config['roc_auc']:.4f}")


Training the model with 2 hidden layers and 16 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 2, Neuronas: 16 -> Loss: 0.6945, Accuracy: 0.4753
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ROC AUC: 0.4892

Training the model with 2 hidden layers and 32 neurons per layer


...

Capas: 8, Neuronas: 64 -> Loss: 0.6954, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
ROC AUC: 0.5218

Training the model with 8 hidden layers and 128 neurons per layer


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Capas: 8, Neuronas: 128 -> Loss: 0.6947, Accuracy: 0.4731
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ROC AUC: 0.4970

Better configuration finded:
Hidden Layers: 3
Neuron(s) per layer: 32
Accuracy: 0.5336
Loss: 0.6930
ROC AUC: 0.5326


In [52]:
# Evaluate on the test set
test_loss, test_accuracy = model_lag.evaluate(X_test_Elkulako_2, Y_test_Elkulako_2)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict probabilities for the positive class
y_pred_probs = model_lag.predict(X_test_Elkulako_2)

# Use df_y_test directly as y_true if it's already binary (0 or 1)
y_true =  Y_test_Elkulako_2  # Assuming df_y_test is not one-hot encoded

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_true, y_pred_probs)
print(f"ROC AUC: {roc_auc}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4802 - loss: 0.6944 
Test Loss: 0.6947478652000427
Test Accuracy: 0.4730941653251648
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
ROC AUC: 0.4969648079056166
